In [55]:
#request the url with id of the film
#Example 
# http://earlycinema.dch.phil-fak.uni-koeln.de/films/view/1
# import the library we use to open URLs
import urllib.request
import re
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup
import pandas as pd


def create_df_film(id):
    # specify which URL/web page we are going to be scraping
    url = "http://earlycinema.dch.phil-fak.uni-koeln.de/films/view/" + str(id)
    # open the url using urllib.request and put the HTML into the page variable
    #catch exception some case id doesn't exist cf 970
    try:
        page = urllib.request.urlopen(url)
        # parse the HTML from our URL into the BeautifulSoup parse tree format
        soup = BeautifulSoup(page, "lxml")
        #the div where the information is
        div_interested = soup.find('div', id='frame1update')
        dateFilms= []
        titles = []
        filmLenghts= []
        filmPricings= []
        filmAllTitles= []
        #filmCountries= []
        filmCompanies= []
        filmGenres= []
        
        ##title + date
        if(div_interested.find('h2', class_='title')):
            title_date = div_interested.find('h2', class_='title').contents[0]


            ##*************Film date***************
            date_film =re.search('(\d+)',title_date)
            if(date_film):
                dateFilms.append(date_film.group())
                ##*************Film title***************
                title = title_date.split("("+date_film.group())
                titles.append(title[0].lstrip().rstrip())
            else:
                dateFilms.append("NO_DATE")
                titles.append(title_date)
        else:
            dateFilms.append("NO_DATE")
            titles.append("NO_TITLE")
        #****************Film Lenght*************************
        #can have more than one parameter 
        #example film id=6
        #<ul>
        #      <li>
        #    4000 Bilder [Bilderzahl entspricht 77_m]              </li>
        #      <li>
        #    80 Meter [Bilderzahl entspricht 77_m]              </li>
        #  </ul>
        lenght = ""
        film_lenght_element = [tag for tag in div_interested.find_all('h3') if tag.text.lstrip().rstrip() == "Film Length"]
        if(film_lenght_element):
            ul_film_lenght = film_lenght_element[0].next_sibling.next_sibling
            if(ul_film_lenght.name == 'ul'):
                for li in ul_film_lenght.findAll('li'):
                    if li.find('ul'):
                        break
                    lenght=lenght +"*"+ li.text.lstrip().rstrip()
                filmLenghts.append(lenght)       
        else:
            filmLenghts.append("") 
        #*********************Pricing**************************
        pricing=""
        film_pricing_element = [tag for tag in div_interested.find_all('h3') if tag.text.lstrip().rstrip() == "Pricing"]
        if(film_pricing_element):
            ul_film_pricing = film_pricing_element[0].next_sibling.next_sibling
            if(ul_film_pricing.name == 'ul'):
                for li in ul_film_pricing.findAll('li'):
                    if li.find('ul'):
                        break
                    pricing=pricing +"*"+ li.text.lstrip().rstrip()
                filmPricings.append(pricing)  
        else:       
            filmPricings.append("") 
        #*******************All titles**********************
        title=""
        film_alltitle_element = [tag for tag in div_interested.find_all('h3') if tag.text.lstrip().rstrip() == "Film titles"]
        if(film_alltitle_element):    
            ul_film_alltitle = film_alltitle_element[0].next_sibling.next_sibling
            if(ul_film_alltitle.name == 'ul'):
                for li in ul_film_alltitle.findAll('li'):
                    if li.find('ul'):
                        break
                    for span in li.findAll('span'):
                        title=title +"*"+ span.text.lstrip().rstrip()
                filmAllTitles.append(title)  
        else:
            filmAllTitles.append("") 

        #*******************Companies - Countries******************
        companie=""
        film_companie_element = [tag for tag in div_interested.find_all('h3') if tag.text.lstrip().rstrip() == "Companies"]
        if(film_companie_element):

            ul_film_companie = film_companie_element[0].next_sibling.next_sibling
            #print(ul_film_companie)
            if(ul_film_companie.name == 'ul'):        
                for a in ul_film_companie.findAll('a'):
                    if a.find('ul'):
                        break
                    film_country =re.search('{.*}',a.text.lstrip().rstrip())
                    #if(film_country):
                    #    filmCountries.append(film_country.group().replace('{',"").replace('}',""))
                    #else:
                    #    filmCountries.append("")
                    companie = companie + "*" + a.text.lstrip().rstrip()
                filmCompanies.append(companie)  
        else:
            filmCompanies.append("") 
            #filmCountries.append("")
        #*****************Film Genres*******************
        genre=''
        film_genre_element = [tag for tag in div_interested.find_all('h3') if tag.text.lstrip().rstrip() == "Genres"]
        if(film_genre_element):
            ul_film_genre = film_genre_element[0].next_sibling.next_sibling
            if(ul_film_genre.name == 'ul'):    
                for a in ul_film_genre.findAll('a'):
                    if a.find('ul'):
                        break
                    genre=genre +"*"+ a.text.lstrip().rstrip()
                try:
                    filmGenres.append(genre)
                except(ValueError):
                    filmGenres.append("ERROR")  
        else:
            filmGenres.append("") 

        df=pd.DataFrame(dateFilms,columns=['Date'])
        df['id early database']=str(id)
        df['Title']=titles
        df['Companies']=filmCompanies
       # df['Countries']=filmCountries
        df['Film lenght']=filmLenghts
        df['Film pricing']=filmPricings
        df['Genre']=filmGenres
        df['All titles']=filmAllTitles
        return df
    except urllib.error.URLError as e:
        print(e.reason + "for id "  + str(id))
    

In [53]:
#film with more than one genre
#id 75
create_df_film(970)


The HTTP server returned a redirect error that would lead to an infinite loop.
The last 30x error message was:
Moved Temporarilyfor id 970


In [51]:
create_df_film(6)


,Date,id early database,Title,Companies,Film lenght,Film pricing,Genre,All titles
0,1901,6,Der Abbruch des historischen Star-Theaters in ...,*k.Ang. (?),*4000 Bilder [Bilderzahl entspricht 77_m]*80 M...,,,*Further Title:*Der Abbruch des berühmten Star...


In [56]:
create_df_film(17760)


,Date,id early database,Title,Companies,Film lenght,Film pricing,Genre,All titles
0,NO_DATE,17760,NO_TITLE,,,,,


In [4]:
create_df_film(4)

,Date,id early database,Title,Companies,Film lenght,Film pricing,Genre,All titles
0,1911,4,"Abaka-Industrie auf der Insel Cebu, Philippinen","*Pathé Frères, Paris (s.a. Thepa) {fr} <Pathe>...",,,*Nicht-fiktional,*Original Title:*Industrie de l' abaca à l' îl...


In [29]:
import time
from datetime import datetime
df = pd.DataFrame(columns=['Date', 'id early database', 'Title','Companies',
    'Countries',
    'Film lenght',
    'Film pricing',
    'Genre',
    'All titles'])


print("start",datetime.now())   
for i in range(10000,20000): 
    try:
        df=df.append(create_df_film(i),sort=False,ignore_index=True)
    except:
        print("problem with id" , str(i))
        raise
df.to_csv('export_early_cinema_10000_20000.csv')

print("finish",datetime.now())

start 2019-10-24 21:58:28.959863
The HTTP server returned a redirect error that would lead to an infinite loop.
The last 30x error message was:
Moved Temporarilyfor id 15218
problem with id 17760


AttributeError: 'NoneType' object has no attribute 'contents'

In [57]:
df

,Date,id early database,Title,Companies,Countries,Film lenght,Film pricing,Genre,All titles
0,1909,1,A B C,*Deutsche Bioscop (fr.:Greenb) (s.a. Biosc) <D...,NaN,*90 Meter,,,
1,1910,2,Aale,"*Elge Gaumont, Leo (Dt.Gaumont) {fr} (s.a. Blu...",NaN,*62 Meter,"*62,00 Mark*4,90 Mark (Surcharge for Colour)",*Nicht-fiktional,*Original Title:*Petites anguilles
2,1910,3,Ab vom rechten Wege,*GB England ohne Produktionsangabe {uk} (Großb...,NaN,*215 Meter,"*215,00 Mark*17,00 Mark (Surcharge for Colour)",,
3,1911,4,"Abaka-Industrie auf der Insel Cebu, Philippinen","*Pathé Frères, Paris (s.a. Thepa) {fr} <Pathe>...",NaN,,,*Nicht-fiktional,*Original Title:*Industrie de l' abaca à l' îl...
4,1901,5,Das Abbrennen des Kaiserpalastes in China,"*Société Générale des Cinemas et Films, Paris ...",NaN,,,*Nicht-fiktional,*Further Title:*Tod des Generals Schwartzkopf
5,1901,6,Der Abbruch des historischen Star-Theaters in ...,*k.Ang. (?),NaN,*4000 Bilder [Bilderzahl entspricht 77_m]*80 M...,,,*Further Title:*Der Abbruch des berühmten Star...
6,1911,7,Der Abdruck des Daumens,*Vitagraph Co.of America {us} <Vitagr> (USA),NaN,,,,*Original Title:*The Thumb Print
7,1911,8,Ein Abend auf dem Bauernhof,*k.Ang. (?)*Deutsche Filmverleiher Vereinigung...,NaN,*218 Meter,"*218,00 Mark*16,00 Mark (Surcharge for Colour)",*Drama,
8,1911,9,Abend auf dem Lande,*Komet Film (fr.:PaulUn) (Deutschland),NaN,*50 Meter,"*50,00 Mark*5,00 Mark (Surcharge for Colour)*5...",*Tonbild,
9,1907,10,Das Abendbrot,*Nordisk Films Co {dk} <Nordsk> (Dänemark),NaN,*65 Meter,"*65,00 Mark",*Lustspiel,*Original Title:*Ripp og Rapp. Aftensmaden*Fur...


In [ ]:
test4

In [ ]:
df.to_csv('export_early_cinema_99.csv')